<a href="https://colab.research.google.com/github/shumshersubashgautam/Deep-NLP/blob/main/NLP_EDA_in_PySpark_%2B_ML_pipeline_LR_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
# from google.colab import files

# files.upload()

In [2]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 74 May  1 08:13 kaggle.json


In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!pwd

/content


In [10]:
!kaggle datasets download -d samdeeplearning/deepnlp

  0% 0.00/234k [00:00<?, ?B/s]
100% 234k/234k [00:00<00:00, 128MB/s]


In [11]:
!unzip /content/deepnlp.zip

Archive:  /content/deepnlp.zip
  inflating: Sheet_1.csv             
  inflating: Sheet_2.csv             


In [12]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=ada2ca15cbf3b889a58a5091c1ecd7afb3487dfa7f22bc97fec5c69160558132
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [13]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from pyspark.sql import SparkSession
SPARK = SparkSession.builder.appName("ProyectoNLPpysparkEDA").getOrCreate()

In [15]:
from pyspark.ml.feature import (Tokenizer, RegexTokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer,NGram)

In [16]:
from pyspark.sql.functions import col,udf, length

In [17]:
from pyspark.sql.types import IntegerType

In [18]:
df = SPARK.read.csv("/content/Sheet_1.csv",
                   inferSchema=True, sep=",",header=True)

In [19]:
df.show()
## Drop the columns without data

+-----------+-----------+--------------------+----+----+----+----+----+
|response_id|      class|       response_text| _c3| _c4| _c5| _c6| _c7|
+-----------+-----------+--------------------+----+----+----+----+----+
| response_1|not_flagged|I try and avoid t...|null|null|null|null|null|
| response_2|    flagged|Had a friend open...|null|null|null|null|null|
| response_3|    flagged|I saved a girl fr...|null|null|null|null|null|
| response_4|not_flagged|i cant think of o...|null|null|null|null|null|
| response_5|not_flagged|"Only really one ...|    |null|null|null|null|
| response_6|not_flagged|a couple of years...|null|null|null|null|null|
| response_7|    flagged|Roommate when he ...|null|null|null|null|null|
| response_8|    flagged|i've had a couple...|null|null|null|null|null|
| response_9|not_flagged|Listened to someo...|null|null|null|null|null|
|response_10|    flagged|I will always lis...|null|null|null|null|null|
|response_11|not_flagged|Took a week off w...|null|null|null|nul

In [20]:
df = df.select("response_id","class","response_text")
## Select only the columns with data

In [21]:
## Let's see the data distribution
## Flagged messages are half the not_flagged

df.groupBy().pivot("class").count().show()

+-------+-----------+
|flagged|not_flagged|
+-------+-----------+
|     25|         55|
+-------+-----------+



In [22]:
df =  df.withColumn("length", length(df["response_text"]))

In [23]:
df.groupBy("class").mean().show()

+-----------+------------------+
|      class|       avg(length)|
+-----------+------------------+
|    flagged|            261.64|
|not_flagged|117.81818181818181|
+-----------+------------------+



In [24]:
df.show(3)

+-----------+-----------+--------------------+------+
|response_id|      class|       response_text|length|
+-----------+-----------+--------------------+------+
| response_1|not_flagged|I try and avoid t...|    37|
| response_2|    flagged|Had a friend open...|   122|
| response_3|    flagged|I saved a girl fr...|   130|
+-----------+-----------+--------------------+------+
only showing top 3 rows



In [25]:
df.select("response_text").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|response_text                                                                                                                                                                                                                                                                                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
## To count words and stopwords we need to tokenize first the messages
tokenizer = Tokenizer(inputCol="response_text" ,outputCol="token_text") 
tokenized = tokenizer.transform(df)

In [27]:
tokenized.show(3)

+-----------+-----------+--------------------+------+--------------------+
|response_id|      class|       response_text|length|          token_text|
+-----------+-----------+--------------------+------+--------------------+
| response_1|not_flagged|I try and avoid t...|    37|[i, try, and, avo...|
| response_2|    flagged|Had a friend open...|   122|[had, a, friend, ...|
| response_3|    flagged|I saved a girl fr...|   130|[i, saved, a, gir...|
+-----------+-----------+--------------------+------+--------------------+
only showing top 3 rows



In [28]:
from pyspark.sql.functions import col, count, explode

Q_of_words = tokenized.select("*", explode("token_text").alias("exploded")).groupBy("class", "response_text","token_text","length").agg(count("exploded").alias("Words"))

In [29]:
Q_of_words.show(3)
## Quiantity of words

+-----------+--------------------+--------------------+------+-----+
|      class|       response_text|          token_text|length|Words|
+-----------+--------------------+--------------------+------+-----+
|not_flagged|I have helped adv...|[i, have, helped,...|    73|   12|
|not_flagged|Listened to someo...|[listened, to, so...|    99|   13|
|    flagged|One of my best fr...|[one, of, my, bes...|   284|   59|
+-----------+--------------------+--------------------+------+-----+
only showing top 3 rows



In [30]:
SWS = StopWordsRemover(inputCol="token_text" ,outputCol="SWS_") 
swsd = SWS.transform(Q_of_words)

In [31]:
swsd.show(3)

+-----------+--------------------+--------------------+------+-----+--------------------+
|      class|       response_text|          token_text|length|Words|                SWS_|
+-----------+--------------------+--------------------+------+-----+--------------------+
|not_flagged|I have helped adv...|[i, have, helped,...|    73|   12|[helped, advise, ...|
|not_flagged|Listened to someo...|[listened, to, so...|    99|   13|[listened, someon...|
|    flagged|One of my best fr...|[one, of, my, bes...|   284|   59|[one, best, frien...|
+-----------+--------------------+--------------------+------+-----+--------------------+
only showing top 3 rows



In [32]:
Q_stops = swsd.select("*", explode("SWS_").alias("boom")).groupBy("class","response_text", "SWS_","Words","length").agg(count("boom").alias("q_stops"))

In [33]:
Q_stops.show(3)
## Quantity of words without StopWords

+-----------+--------------------+--------------------+-----+------+-------+
|      class|       response_text|                SWS_|Words|length|q_stops|
+-----------+--------------------+--------------------+-----+------+-------+
|    flagged|I've had some fri...|[friends, come, s...|   50|   276|     23|
|not_flagged|I've always been ...|[always, good, li...|   60|   312|     28|
|    flagged|One of my best fr...|[one, best, frien...|   59|   284|     23|
+-----------+--------------------+--------------------+-----+------+-------+
only showing top 3 rows



In [34]:
Q_stops.groupBy("class").mean().show()

+-----------+-----------------+------------------+------------------+
|      class|       avg(Words)|       avg(length)|      avg(q_stops)|
+-----------+-----------------+------------------+------------------+
|    flagged|             50.6|            261.64|              22.8|
|not_flagged|22.69090909090909|117.81818181818181|10.545454545454545|
+-----------+-----------------+------------------+------------------+



In [35]:
Q_of_words.groupBy("class").mean().show()
# Flagged have more words

+-----------+------------------+-----------------+
|      class|       avg(length)|       avg(Words)|
+-----------+------------------+-----------------+
|    flagged|            261.64|             50.6|
|not_flagged|117.81818181818181|22.69090909090909|
+-----------+------------------+-----------------+



In [36]:
## In order to get the Q of stopwords we substract Q_WORDS- Q_STOPWOWRDS
df2 = Q_stops.withColumn("StopWordsQ",col("Words")-col("q_stops"))

In [37]:
df2.show(3)

+-----------+--------------------+--------------------+-----+------+-------+----------+
|      class|       response_text|                SWS_|Words|length|q_stops|StopWordsQ|
+-----------+--------------------+--------------------+-----+------+-------+----------+
|    flagged|I've had some fri...|[friends, come, s...|   50|   276|     23|        27|
|not_flagged|I've always been ...|[always, good, li...|   60|   312|     28|        32|
|    flagged|One of my best fr...|[one, best, frien...|   59|   284|     23|        36|
+-----------+--------------------+--------------------+-----+------+-------+----------+
only showing top 3 rows



In [38]:
df2.groupBy("class").mean().show()
## Flagged have more stopwords

+-----------+-----------------+------------------+------------------+------------------+
|      class|       avg(Words)|       avg(length)|      avg(q_stops)|   avg(StopWordsQ)|
+-----------+-----------------+------------------+------------------+------------------+
|    flagged|             50.6|            261.64|              22.8|              27.8|
|not_flagged|22.69090909090909|117.81818181818181|10.545454545454545|12.145454545454545|
+-----------+-----------------+------------------+------------------+------------------+



In [39]:
### Create the variables for the pipeline
tokenizer = Tokenizer(inputCol="response_text" ,outputCol="token_text")
stop_remove=StopWordsRemover(inputCol="token_text" ,outputCol="stop_text")
count_vect=CountVectorizer(inputCol="stop_text" ,outputCol="c_vec")
idf=IDF(inputCol="c_vec" ,outputCol="tf_idf")

flag_nflag_numeric=StringIndexer(inputCol="class" ,outputCol="label") ### Label Encoder ( 0, 1)

In [40]:
from pyspark.ml.feature import VectorAssembler

In [41]:
## Choose the features
clean_up = VectorAssembler(inputCols=["tf_idf","length","StopWordsQ"],outputCol="features")

In [42]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LogisticRegression

In [43]:
nb  = NaiveBayes()
lr = LogisticRegression()

In [44]:
from pyspark.ml import Pipeline

In [45]:
## Create the pipeline
data_prep_pipe = Pipeline(stages = [flag_nflag_numeric,tokenizer,stop_remove,count_vect,
                          idf,clean_up])

In [46]:
cleaner = data_prep_pipe.fit(df2)

In [47]:
clean_data = cleaner.transform(df2)

In [48]:
clean_data = clean_data.select("label","features")

In [49]:
clean_data.show(4)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(661,[0,1,2,9,14,...|
|  0.0|(661,[2,3,12,15,1...|
|  1.0|(661,[1,4,8,11,14...|
|  0.0|(661,[1,3,9,14,17...|
+-----+--------------------+
only showing top 4 rows



In [50]:
training,test=clean_data.randomSplit([0.7,0.3])

In [51]:
flag_detector = nb.fit(training)

In [52]:
test_results = flag_detector.transform(test)

In [53]:
test_results.show(3)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(661,[0,3,8,36,48...|[-1184.4708210377...|[1.04049506505653...|       1.0|
|  0.0|(661,[0,6,19,35,3...|[-193.80609579086...|[0.99994296873365...|       0.0|
|  0.0|(661,[0,55,88,544...|[-115.79675779250...|[0.01333346952558...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 3 rows



In [54]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval=MulticlassClassificationEvaluator()

In [55]:
f1 = acc_eval.evaluate(test_results,{acc_eval.metricName: "f1"})

In [56]:
print(f1)

0.4839506172839507


In [57]:
evaluator= MulticlassClassificationEvaluator()

evaluator.setPredictionCol("prediction")


evaluator.evaluate(test_results)


evaluator.evaluate(test_results, {evaluator.metricName: "accuracy"})

0.47619047619047616

In [58]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
preds_and_labels = test_results.select(['prediction',
                                       'label']).withColumn('label',
                                                        F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
from pyspark.mllib.evaluation import MulticlassMetrics
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[8. 6.]
 [5. 2.]]


In [59]:
lr_metric = MulticlassMetrics(test_results["label","prediction"].rdd)
print("Accuracy", lr_metric.accuracy)
print("Precision", lr_metric.precision(1.0))
print("Recall", lr_metric.recall(1.0))
print("F1score", lr_metric.fMeasure(1.0))

Accuracy 0.47619047619047616
Precision 0.2857142857142857
Recall 0.25
F1score 0.26666666666666666


In [60]:
training,test=clean_data.randomSplit([0.7,0.3])

In [61]:
flag_detector = lr.fit(training)

In [62]:
test_results = flag_detector.transform(test)

In [63]:
### It predicts almost everything like 0
test_results.show(80)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(661,[0,1,2,6,11,...|[9.70555159991846...|[0.99993905949772...|       0.0|
|  0.0|(661,[0,3,8,36,48...|[-3.7637125660631...|[0.02267153620012...|       1.0|
|  0.0|(661,[1,9,29,33,3...|[16.5887157106814...|[0.99999993753851...|       0.0|
|  0.0|(661,[2,5,27,58,1...|[12.5436895764237...|[0.99999643267000...|       0.0|
|  0.0|(661,[2,26,38,43,...|[13.4540786911899...|[0.99999856461883...|       0.0|
|  0.0|(661,[3,13,30,36,...|[8.17808440885354...|[0.99971933958547...|       0.0|
|  0.0|(661,[3,475,477,6...|[13.1610674676920...|[0.99999807593301...|       0.0|
|  0.0|(661,[4,10,45,58,...|[18.2477098460134...|[0.99999998811168...|       0.0|
|  0.0|(661,[5,20,28,179...|[10.4807512949252...|[0.99997192916346...|       0.0|
|  0.0|(661,[5,2

In [64]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval=MulticlassClassificationEvaluator()

In [65]:
f1 = acc_eval.evaluate(test_results,{acc_eval.metricName: "f1"})

In [66]:
print(f1)

0.5979020979020979


In [67]:
evaluator= MulticlassClassificationEvaluator()

evaluator.setPredictionCol("prediction")


evaluator.evaluate(test_results)


evaluator.evaluate(test_results, {evaluator.metricName: "accuracy"})

0.6923076923076923

In [68]:
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
preds_and_labels = test_results.select(['prediction',
                                       'label']).withColumn('label',
                                                        F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
from pyspark.mllib.evaluation import MulticlassMetrics
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

[[18.  1.]
 [ 7.  0.]]


In [69]:
lr_metric = MulticlassMetrics(test_results["label","prediction"].rdd)
print("Accuracy", lr_metric.accuracy)
#print("Precision", lr_metric.precision(1.0))
#print("Recall", lr_metric.recall(1.0))
#print("F1score", lr_metric.fMeasure(1.0))

# Right now it throwed an error on Precision, Recall and F1score. Because the 0's from the previous cell 
# as you can see the Confusion Matrix throws two 0's on the second column, which means the model is predicting
# everything as 0's in the model. If you run this code multiple time you will find that sometimes
# it will predict everything as 0 or very few as 1. You can also use a seed. 

Accuracy 0.6923076923076923
